In [1]:
import pandas
pandas.set_option('max_colwidth', 200)

In [2]:
from corus import load_lenta
path = 'data/lenta-ru-news.csv.gz'
records = load_lenta(path)
next(records)

LentaRecord(
    url='https://lenta.ru/news/2018/12/14/cancer/',
    title='Названы регионы России с\xa0самой высокой смертностью от\xa0рака',
    text='Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.',
    topic='Россия',
    tags='Общество'
)

In [3]:
#preprocessing: lower-case, commas, lemmatization
#PRACTICE: implement preprocessing

In [4]:
import razdel
from pymystem3 import Mystem
#from pymorphy2 import MorphAnalyzer

_LEMMATIZER = None
def tag(word):
    global _LEMMATIZER
    if _LEMMATIZER is None:
        _LEMMATIZER = Mystem()
        #_LEMMATIZER = MorphAnalyzer('ru')
    try:
        processed = _LEMMATIZER.analyze(word)[0]
    except:
        return word
    if not processed.get('analysis'):
        return word.strip().lower()
    lemma = processed["analysis"][0]["lex"].lower().strip()
    return lemma

import re
def tokenize_re(doc):
    return [tag(x) for x in re.split('(\W+)', doc) if x.strip()]

def tokenize(doc):
    # упомянуть регулярные выражения
    return [tag(t.text) for t in razdel.tokenize(doc)]

import sentencepiece
def tokenize_bpe(doc):
    #FIXME
    pass

print(tokenize_re('Выплаты на второго ребёнка'))
print(tokenize('Выплаты на второго ребёнка'))

['выплата', 'на', 'второй', 'ребенок']
['выплата', 'на', 'второй', 'ребенок']


In [5]:
print(tokenize_re('Названы регионы России с\xa0самой высокой смертностью, от\xa0рака... Привет всем!!!'))
print(tokenize('Названы регионы России с\xa0самой высокой смертностью, от\xa0рака... Привет всем!!!'))
print(tokenize_re('Это на 3,5 процента меньше, чем годом ранее.'))
print(tokenize('Это на 3,5 процента меньше, чем годом ранее.'))

['называть', 'регион', 'россия', 'с', 'самый', 'высокий', 'смертность', ',', 'от', 'рак', '...', 'привет', 'все', '!!!']
['называть', 'регион', 'россия', 'с', 'самый', 'высокий', 'смертность', ',', 'от', 'рак', '...', 'привет', 'все', '!!!']
['это', 'на', '3', ',', '5', 'процент', 'мало', ',', 'что', 'год', 'ранее', '.']
['это', 'на', '3,5', 'процент', 'мало', ',', 'что', 'год', 'ранее', '.']


In [6]:
import pandas
from tqdm import tqdm_notebook as tq

records = load_lenta(path)
# overfitting and underfitting
# splitting the dataset. how many examples to take for a test?
dataset = []
for i, r in tq(enumerate(records), total=25000):
    if not r.topic:
        # есть примеры в датасете, где нет категории
        continue
    dataset.append((tokenize(r.title), r.topic))
    if len(dataset) >= 25000:
        break
df = pandas.DataFrame(dataset)
print(len(df))
df.head(5)

25000


,0,1
0,"[называть, регион, россия, с, самый, высокий, смертность, от, рак]",Россия
1,"[австрия, не, представлять, доказательство, вина, российский, биатлонист]",Спорт
2,"[обнаруживать, самый, счастливый, место, на, планета]",Путешествия
3,"[в, сша, раскрывать, сумма, расход, на, расследование, «, российский, дело, »]",Мир
4,"[хакер, рассказывать, о, план, великобритания, заминировать, севастополь]",Мир


In [7]:
classes = sorted(set([_[1] for _ in dataset]))
print(len(classes), classes)

17 ['69-я параллель', 'Бизнес', 'Бывший СССР', 'Дом', 'Из жизни', 'Интернет и СМИ', 'Крым', 'Культпросвет ', 'Культура', 'Мир', 'Наука и техника', 'Путешествия', 'Россия', 'Силовые структуры', 'Спорт', 'Ценности', 'Экономика']


Dice with N sides STD formula: <img src="files/dice-std.png">

In [8]:
# Calculator: https://select-statistics.co.uk/calculators/sample-size-calculator-two-proportions/
import math
def std_n_classes(n):
    return math.sqrt((n*n-1) / 12)
display(
    std_n_classes(2),
    std_n_classes(17),
    std_n_classes(17) / std_n_classes(2),
    24638 / 9.797958)

0.5

4.898979485566356

9.797958971132712

2514.6055943493534

In [9]:
#train, dev, test
#20000, 2500, 2500 
from sklearn.model_selection import train_test_split
X_train, X_dev, y_train, y_dev = train_test_split([_[0] for _ in dataset], 
                                                  [_[1] for _ in dataset], test_size=2500, random_state=42)
display(pandas.DataFrame([[x] for x in X_train[:5]], columns=['x_train']), 
        pandas.DataFrame(y_train[:5], columns=['y_train']))
X_train, X_test, y_train, y_test = train_test_split(X_train, 
                                                    y_train, test_size=2500, random_state=42)

,x_train
0,"[центробанк, впервые, рассказывать, о, теневой, схема]"
1,"[семь, человек, сходить, на, музыкальный, фестиваль, и, умирать]"
2,"[россиянин, становиться, работать, за, долг]"
3,"[группа, little, big, представлять, новый, альбом, в, москва]"
4,"[получать, первый, в, история, фото, с, поверхность, астероид]"


,y_train
0,Экономика
1,Культура
2,Экономика
3,Культура
4,Наука и техника


In [10]:
display(pandas.DataFrame([[x] for x in X_dev[:5]], columns=['x_dev']), 
        pandas.DataFrame(y_dev[:5], columns=['y_dev']))

,x_dev
0,"[телеведущая, отказывать, в, работа, из-за, «, недостаточно, большой, грудь, »]"
1,"[шарапов, отвечать, на, отказ, серена, уильямс, играть, против, она]"
2,"[bvlgari, представлять, новый, вариация, на, змеиный, тема]"
3,"[находить, новый, способ, побеждать, особо, опасный, рак]"
4,"[анестезиолог, изнасиловать, россиянка, под, наркоз, и, пойти, под, суд]"


,y_dev
0,Интернет и СМИ
1,Спорт
2,Ценности
3,Наука и техника
4,Силовые структуры


In [11]:
display(len(X_train), len(y_train), len(X_dev), len(y_dev), len(X_test), len(y_test))

20000

20000

2500

2500

2500

2500

In [12]:
TEST_ITEMS = {
    6: 'Это первое предложение. Это оно снова...', 
    8: 'Это пример второго предложения, ха-ха-ха',
    9: 'А это третье предложение',
    10: 'выплата на ребенка'}
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 2), tokenizer=tokenize)
vectorizer.fit_transform(TEST_ITEMS.values())
import pprint; pprint.pprint(list(enumerate(vectorizer.get_feature_names())))

[(0, ','),
 (1, ', ха-ха'),
 (2, '.'),
 (3, '. это'),
 (4, '...'),
 (5, 'а'),
 (6, 'а это'),
 (7, 'второй'),
 (8, 'второй предложение'),
 (9, 'выплата'),
 (10, 'выплата на'),
 (11, 'на'),
 (12, 'на ребенок'),
 (13, 'оно'),
 (14, 'оно снова'),
 (15, 'первый'),
 (16, 'первый предложение'),
 (17, 'предложение'),
 (18, 'предложение ,'),
 (19, 'предложение .'),
 (20, 'пример'),
 (21, 'пример второй'),
 (22, 'ребенок'),
 (23, 'снова'),
 (24, 'снова ...'),
 (25, 'третий'),
 (26, 'третий предложение'),
 (27, 'ха-ха'),
 (28, 'это'),
 (29, 'это оно'),
 (30, 'это первый'),
 (31, 'это пример'),
 (32, 'это третий')]


In [13]:
tokenized = tokenize_re('Выплаты на второго ребёнка')
print(tokenized)
feats_example = vectorizer.transform(tokenized)
print("Example:")
print(feats_example)
print("Dense:")
print(feats_example.todense())

['выплата', 'на', 'второй', 'ребенок']
Example:
  (0, 9)	1
  (1, 11)	1
  (2, 7)	1
  (3, 22)	1
Dense:
[[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]]


In [14]:
TEST_QUERY = 'ха-ха, первое предложение для примера!'
print("Test example:")
feats_test = vectorizer.transform(tokenize(TEST_QUERY))
print(feats_test)
print("Test dense:")
print("Dense:")
print(feats_test.todense())

Test example:
  (0, 27)	1
  (1, 0)	1
  (2, 15)	1
  (3, 17)	1
  (5, 20)	1
Test dense:
Dense:
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [15]:
vec_x = CountVectorizer(ngram_range=(1, 2), tokenizer=lambda x: x, lowercase=False, min_df=1)
vec_x.fit_transform(X_train)
import pprint; pprint.pprint(list(enumerate(vec_x.get_feature_names()[::10000])))
print("Vocabulary size:", len(vec_x.vocabulary_))

[(0, '!'),
 (1, 'британский разведка'),
 (2, 'дворцовый переворот'),
 (3, 'игра с'),
 (4, 'между мат'),
 (5, 'обнаженный азия'),
 (6, 'позволять'),
 (7, 'разрушать стадион'),
 (8, 'соглашаться'),
 (9, 'факт »')]
Vocabulary size: 95391


In [16]:
vec_x = CountVectorizer(ngram_range=(1, 2), tokenizer=lambda x: x, lowercase=False, min_df=2)
vec_x.fit_transform(X_train)
import pprint; pprint.pprint(list(enumerate(vec_x.get_feature_names()[::10000])))
print("Vocabulary size:", len(vec_x.vocabulary_))

[(0, '!'), (1, 'машина и'), (2, 'совместно')]
Vocabulary size: 23708


In [17]:
# 20000 x 17000

In [18]:
vec_y = {c: i for i, c in enumerate(classes)}
vec_y

{'69-я параллель': 0,
 'Бизнес': 1,
 'Бывший СССР': 2,
 'Дом': 3,
 'Из жизни': 4,
 'Интернет и СМИ': 5,
 'Крым': 6,
 'Культпросвет ': 7,
 'Культура': 8,
 'Мир': 9,
 'Наука и техника': 10,
 'Путешествия': 11,
 'Россия': 12,
 'Силовые структуры': 13,
 'Спорт': 14,
 'Ценности': 15,
 'Экономика': 16}

In [19]:
train_x_vec = vec_x.transform(X_train)
dev_x_vec = vec_x.transform(X_dev)

In [20]:
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=0, tol=1e-5)
clf.fit(train_x_vec, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=0, tol=1e-05,
          verbose=0)

In [21]:
predict_dev = clf.predict(dev_x_vec)
pandas.DataFrame(zip(predict_dev, y_dev, X_dev), columns=['predicted', 'expected', 'title'])

,predicted,expected,title
0,Интернет и СМИ,Интернет и СМИ,"[телеведущая, отказывать, в, работа, из-за, «, недостаточно, большой, грудь, »]"
1,Спорт,Спорт,"[шарапов, отвечать, на, отказ, серена, уильямс, играть, против, она]"
2,Ценности,Ценности,"[bvlgari, представлять, новый, вариация, на, змеиный, тема]"
3,Наука и техника,Наука и техника,"[находить, новый, способ, побеждать, особо, опасный, рак]"
4,Россия,Силовые структуры,"[анестезиолог, изнасиловать, россиянка, под, наркоз, и, пойти, под, суд]"
...,...,...,...
2495,Спорт,Спорт,"[смалывать, поведывать, о, смерть, на, футбольный, поле]"
2496,Мир,Интернет и СМИ,"[plus-size-блогер, рассказывать, о, нелепый, комплимент, мужчина]"
2497,Интернет и СМИ,Экономика,"[оператор, находить, способ, частично, окупать, «, закон, яровой, »]"
2498,Россия,Россия,"[россиянка, давать, отпор, избивать, она, козленок, полицейский, и, быть, наказывать]"


In [22]:
clf.score(dev_x_vec, y_dev)

0.7248

In [23]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [24]:
print(classification_report(predict_dev, y_dev, digits=3))

                   precision    recall  f1-score   support

   69-я параллель      0.545     0.923     0.686        13
      Бывший СССР      0.789     0.759     0.774       158
              Дом      0.768     0.768     0.768        82
         Из жизни      0.654     0.651     0.652       186
   Интернет и СМИ      0.595     0.685     0.637       165
             Крым      0.000     0.000     0.000         0
         Культура      0.744     0.726     0.735       164
              Мир      0.726     0.689     0.707       347
  Наука и техника      0.798     0.753     0.775       178
      Путешествия      0.662     0.707     0.684        75
           Россия      0.695     0.644     0.668       385
Силовые структуры      0.625     0.616     0.620       138
            Спорт      0.920     0.906     0.913       266
         Ценности      0.713     0.750     0.731        96
        Экономика      0.698     0.777     0.736       247

         accuracy                          0.725      

/Projects/ai/nlpworkshop/.venv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [25]:
print(confusion_matrix(predict_dev, y_dev))

[[ 12   0   0   1   0   0   0   0   0   0   0   0   0   0   0]
 [  0 120   0   0   4   0   4   2   3   1   6   2   1   1  14]
 [  0   1  63   2   1   0   0   2   0   0   5   4   0   0   4]
 [  2   1   2 121   8   0   3  21   2   3   9   4   3   5   2]
 [  1   0   0  10 113   0   5   7   1   5   8   1   4   3   7]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  3   1   1   5  10   0 119   6   0   2   6   4   2   4   1]
 [  0   8   0  13  13   0   6 239  12   4  24   4   3   4  17]
 [  0   4   0   4   6   1   5   6 134   3   3   1   1   5   5]
 [  0   2   2   3   3   0   1   4   2  53   3   0   0   0   2]
 [  2   7   9   7  13   0  10  23   7   5 248  24   4   2  24]
 [  0   4   2   5   3   0   3   5   0   1  27  85   1   0   2]
 [  0   0   0   1   5   0   2   3   1   1   2   4 241   3   3]
 [  0   0   0   9   4   0   2   4   0   1   0   1   1  72   2]
 [  2   4   3   4   7   0   0   7   6   1  16   2   1   2 192]]


In [26]:
lines = [' '.join(['__label__'+y.replace(' ','_')] + x) for x, y in zip(X_train, y_train)]
lines[:5]
from pathlib import Path
Path('data/train.txt').write_text('\n'.join(lines))

1466148

In [27]:
lines = [' '.join(['__label__'+y] + x) for x, y in zip(X_dev, y_dev)]
Path('data/dev.txt').write_text('\n'.join(lines))

183526

In [28]:
!dist/fastText-0.9.1/fasttext supervised -input data/train.txt -output data/ft.bin -minCount 2 -epoch 10 -wordNgrams 1

Read 0M words
Number of words:  8505
Number of labels: 17
Progress: 100.0% words/sec/thread:  523823 lr:  0.000000 loss:  0.888096 ETA:   0h 0m


In [29]:
!dist/fastText-0.9.1/fasttext test data/ft.bin.bin data/dev.txt

N	1647
P@1	0.733
R@1	0.733


In [30]:
import fasttext
model = fasttext.load_model('data/ft.bin.bin')

In [31]:
X_dev[0], y_dev[0]

(['телеведущая',
  'отказывать',
  'в',
  'работа',
  'из-за',
  '«',
  'недостаточно',
  'большой',
  'грудь',
  '»'],
 'Интернет и СМИ')

In [32]:
display(list(zip(*model.predict(' '.join(X_dev[0]), k=1))))
predicted = []
for x in tq(X_dev):
    predicts = model.predict(' '.join(x), k=1)[0][0]
    #print(predicts)
    predicted.append(vec_y[predicts[9:].replace('_', ' ')])
    expected = [vec_y[y] for y in y_dev]
display(predicted[:10], expected[:10])
accuracy_score(predicted, expected)

[('__label__Интернет_и_СМИ', 0.8638997077941895)]

[5, 14, 15, 10, 13, 12, 10, 12, 2, 4]

[5, 14, 15, 10, 13, 12, 10, 9, 16, 4]

0.7216

In [33]:
# NN
# more complex models
